In [2]:
import pandas as pd
import numpy as np
import os
import glob

# Load Files Here: (extension name needed)
input_folder = 'test/Na_K_NH4_adducts/'
output_folder = 'test/Na_K_NH4_adducts/'
csv_files = glob.glob(os.path.join(input_folder, '*.csv'))
# NIST = pd.read_csv('../PFAS_libraries/NIST_PFAS_Suspect_List_corrected.csv')
NIST = pd.read_csv('../PFAS_libraries/NIST_PFAS_Suspect_List_pos.csv')
    
# Loop through each file in the folder
for input_file_path in csv_files:
    if input_file_path.endswith('_series.csv'):
        continue
    # Input File:
    df = pd.read_csv(input_file_path)
    df['ppm_error'] = df['ppm_error'].round(4)
    df['exper_mz'] = df['exper_mz'].round(5) 
    df.insert(4, 'abund_norm', np.nan)
    df['abund_norm'] = df['abund'] / df['abund'].max()

    # Output File:
    output_file_name_stem = os.path.splitext(os.path.basename(input_file_path))[0] 
    output_file_name = f"{output_file_name_stem.replace('processed', 'series')}.xlsx" 
    output_file = os.path.join(output_folder, output_file_name)
    if os.path.exists(output_file):
        continue
         
    df = df.drop(df.columns[0], axis=1)
    
    """1. Break the input file into 3 subframes: df1 = CF2 series only; df2 = CH2 series only; and df3 = C2H4O series only"""
    dfs = []
    start_index = 0

    for i in range(1, len(df)):
        if i == len(df) - 1:
            dfs.append(df.iloc[start_index:])
            break
        
        diff = abs(df.iloc[i]['exper_mz'] - df.iloc[i-1]['exper_mz'])
        
        if diff >= 500:
            dfs.append(df.iloc[start_index:i])
            start_index = i
    if len(dfs) == 3:
        df1, df2, df3 = dfs
    else:
        print(f"Skipping file {input_file_path} as it does not have the expected number of breakpoints.")
        continue
        
    df1, df2, df3 = dfs
    
    """2. Data cleanup"""
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    df3 = df3.drop_duplicates()
    
    # Please do your own statistics and decide what criteria you should use for better credibility 
    df1 = df1[df1['I'] == 0]
    df1 = df1[df1['F'] >= 5]  
    
    all_iso_values = df1[['13C_iso?', '81Br_iso?', '37Cl_iso?', '18O_iso?', '34S_iso?']].values.flatten()
    df1['isotope?'] = df1['exper_mz'].apply(lambda x: 'Y' if x in all_iso_values else 'N')
    
    # Isotope peaks are removed so we may see some members are gone in a homologous series. Warning: 18-oxygen may have +/- 60%
    # isotopic abundance error so this criteria may or may not remove too many        
    df1 = df1[df1['isotope?'] == 'N']
    df1.drop(columns=['isotope?'])
    
    # This function limits the delta C number to be 1 or 2 only, corresponding to CF2 or C2F4    
    def carbon_number_checker(group):
        sorted_C = sorted(group['C'].unique())  
        differences = [sorted_C[i+1] - sorted_C[i] for i in range(len(sorted_C)-1)]  
        return all(diff in {1, 2} for diff in differences) 
    CF2_series = df1.groupby(['KMD_CF2', 'z*_CF2', 'H', 'Cl', 'Br', 'P', 'S']).filter(carbon_number_checker)
    
    # You can also clean up data for other homologous series such as CH2 or C2H4O: 
    # CH2_series = df2.sort_values(['KMD_CH2', 'z*_CH2', 'F', 'Cl', 'Br', 'P', 'S'])
    # C2H4O_series = df3.sort_values(['KMD_C2H4O', 'z*_C2H4O', 'F', 'Cl', 'Br', 'P', 'S'])

    
    """3. Categorization (confidence levels)""" 
    # Cat A - known PFAS with all members having unique formula in a homologous series (>=3 members); 
    # Cat B - unknown PFAS with all members having unique formula in a homologous series (>=3 members); 
    # Cat C - unknown PFAS with at least 1 member having unique formula in a homologous series (>=3 members); 
    # Cat D - unknown PFAS with all members having multiple assignments but at least 1 member having correct isotopologues;
    # Cat E - no isotope fine structure for any member in a CF2 homologous series - they may be false positive!
    # Note, we will define the length cutoff for each homologous series in Step 4
    
    
    """3-1. Category E (no isotope fine structure for any member in a CF2 homologous series):"""
    CF2_series.insert(13, 'category', np.nan)
    columns_to_check = ["13C_match?", "81Br_match?", "37Cl_match?", "18O_match?", "34S_match?"]

    def to_category_E(group):
        if not (group[columns_to_check] == "Y").any().any():  
            group["category"] = "E"  
        return group
    CF2_series = CF2_series.groupby(['KMD_CF2', 'z*_CF2', 'H', 'Cl', 'Br', 'P', 'S'], group_keys=False).apply(to_category_E)
    
    
    """3-2. Uniform the KMD_CF2 values for different assignments of a peak:"""
    CF2_series.insert(7, 'KMD_CF2_new', np.nan)
    CF2_series['KMD_CF2_new'] = CF2_series['KMD_CF2']
            
              
    """3-3. Correct the KMD values (this is because different assignments of a peak may have slightly different KMD values."""
    ## We need to unify all KMD values of different assignments):  
    CF2_series['KMD_CF2_new'] = CF2_series.groupby('exper_mz')['KMD_CF2'].transform('max')
    CF2_series['KMD_CF2_new'] = CF2_series.groupby(['KMD_CF2', 'z*_CF2'])['KMD_CF2_new'].transform('max')
    CF2_series['KMD_CF2_new'] = CF2_series.groupby('exper_mz')['KMD_CF2_new'].transform('max')
    CF2_series['KMD_CF2_new'] = CF2_series.groupby(['KMD_CF2', 'z*_CF2'])['KMD_CF2_new'].transform('max')
    
            
    """3-4. Categories B, C, D:"""
    ## Add helper columns 'group_mz_count_max' and 'group_mz_count_min'
    CF2_series.insert(7, 'mz_count_max', np.nan)
    CF2_series.insert(7, 'mz_count_min', np.nan)
    CF2_series['mz_count_max'] = CF2_series.groupby(['KMD_CF2_new', 'z*_CF2'])['exper_mz'].transform(lambda x: x.value_counts().max())
    CF2_series['mz_count_min'] = CF2_series.groupby(['KMD_CF2_new', 'z*_CF2'])['exper_mz'].transform(lambda x: x.value_counts().min())
    
    def count_ones(x):
        counts = x.value_counts()
        return sum(counts == 1)
    CF2_series.insert(14, 'mz_count_ones', np.nan)
    CF2_series['mz_count_ones'] = CF2_series.groupby(['KMD_CF2_new', 'z*_CF2'])['exper_mz'].transform(count_ones)

    # The 'conditions' below means: 
    # (1) In a CF2 series, if the max occurence of each m/z (max number of assignments a peak could have) and min occurence of each m/z
    # (min number of assignments a peak could have) are both > 1, this means all members have more than 1 assignment and therefore are Cat D/E. 
    # (2) If the max occurence of each m/z > 1 but min occurence of each m/z = 1, this means at least 1 members have unique assignments whereas
    # other members have more than 1 assignment. In this case, they are all in Cat C.
    # (3) If the max occurence of each m/z and min occurence of each m/z = 1, or only 1 member has unique assignment,
    # this means all members have only 1 assignment and therefore should be Cat B.

    conditions = [
        (CF2_series['mz_count_max'] > 1) & (CF2_series['mz_count_min'] > 1), 
        (CF2_series['mz_count_max'] > 1) & (CF2_series['mz_count_min'] == 1) & (CF2_series['mz_count_ones'] >= 1), 
        (CF2_series['mz_count_max'] == 1), 
    ]
    which_category = ['D', 'C', 'B']
    no_category_E = CF2_series['category'] != 'E'
    new_category = np.select(conditions, which_category, default=CF2_series['category'])
    CF2_series.loc[no_category_E, 'category'] = new_category[no_category_E]
    
    """3-5. Category A:"""
    found_in_NIST = CF2_series['formula'].isin(NIST['formula'])
    CF2_series.loc[found_in_NIST, 'category'] = 'A'
    CF2_series['category'] = CF2_series.groupby(['KMD_CF2', 'z*_CF2', 'H', 'P', 'S'])['category'].transform(lambda x: 'A' if 'A' in x.values else x)    
    # Here groupby(['KMD_CF2', ... should be used instead of groupby(['KMD_CF2_new', ... !
    
    def alt_assgn_of_cat_A(x):
        if 'A' in x.values or 'A' in x.values :
            return x.replace({'B':'A alt assgn', 'C':'A alt assgn', 'D':'A alt assgn', 'E':'A alt assgn'})
        else:
            return x

    CF2_series['category'] = CF2_series.groupby(['exper_mz'])['category'].transform(alt_assgn_of_cat_A)
    CF2_series['category'] = CF2_series.groupby(['KMD_CF2', 'z*_CF2', 'H', 'P', 'S'])['category'].transform(alt_assgn_of_cat_A)
    CF2_series = CF2_series[~CF2_series['category'].isin(['A alt assgn'])]
    
    """4. Keep series with 4 or above members for UNKNOWN PFAS (Cat B, C, D, and E) only"""
    
    # First, Cat B, C, and D:      
    CF2_series_BCD = CF2_series[CF2_series['category'].isin(['B', 'C', 'D'])]
    CF2_series_BCD = CF2_series_BCD.groupby(['KMD_CF2_new', 'z*_CF2', 'H', 'Cl', 'Br', 'P', 'S']).filter(lambda x: len(x) >= 4)
    
    # Then we need to update the categorization of previous Cat C and D homologous series which have 3 members but have been removed  
    # as we now require 4 members. For example, for m/z 500-550-600-650, 500 has 1 assignment while the others have 2. Then the second set of
    # assignment 550-600-650 has been removed because we now only allow series with at least 4 members. Therefore, only the first set of assignment
    # 500-550-600-650 will be remained. 
    # This set, which was previously a Cat C or D series, now needs to be updated to Cat B. 
    # Nothing to do with Cat E because they don't have isotope fine structures!

    CF2_series_BCD['mz_count_max'] = np.nan
    CF2_series_BCD['mz_count_min'] = np.nan
    CF2_series_BCD['mz_count_max'] = CF2_series_BCD.groupby(['KMD_CF2_new', 'z*_CF2'])['exper_mz'].transform(lambda x: x.value_counts().max())
    CF2_series_BCD['mz_count_min'] = CF2_series_BCD.groupby(['KMD_CF2_new', 'z*_CF2'])['exper_mz'].transform(lambda x: x.value_counts().min())
    CF2_series_BCD['mz_count_ones'] = CF2_series_BCD.groupby(['KMD_CF2_new', 'z*_CF2'])['exper_mz'].transform(count_ones)
    conditions = [
        ((CF2_series_BCD['mz_count_max'] > 1) & (CF2_series_BCD['mz_count_min'] > 1)), 
        (CF2_series_BCD['mz_count_max'] > 1) & (CF2_series_BCD['mz_count_min'] == 1) & (CF2_series_BCD['mz_count_ones'] >= 1), 
        (CF2_series_BCD['mz_count_max'] == 1), 
    ]
    CF2_series_BCD['category'] = np.select(conditions, which_category, default='nan')
    
    # Now Cat E:      
    CF2_series_E = CF2_series[CF2_series['category'].isin(['E'])]
    CF2_series_E = CF2_series_E.groupby(['KMD_CF2_new', 'z*_CF2', 'H', 'Cl', 'Br', 'P', 'S']).filter(lambda x: len(x) >= 4)
    
    # Finally piece them together:
    CF2_series = pd.concat([CF2_series[~CF2_series['category'].isin(['B', 'C', 'D', 'E'])], CF2_series_BCD, CF2_series_E])
    number_of_A = CF2_series[CF2_series['category'] == 'A']['KMD_CF2_new'].nunique()
    number_of_B = CF2_series[CF2_series['category'] == 'B']['KMD_CF2_new'].nunique()
    number_of_C = CF2_series[CF2_series['category'] == 'C']['KMD_CF2_new'].nunique()
    number_of_D = CF2_series[CF2_series['category'] == 'D']['KMD_CF2_new'].nunique()
    number_of_E = CF2_series[CF2_series['category'] == 'E']['KMD_CF2_new'].nunique()
    
    """5. Stats, and output of results"""  
    # Drop helper lines first:
    CF2_series = CF2_series.drop(columns=['mz_count_max', 'mz_count_min', 'mz_count_ones', 'KMD_CH2', 'z*_CH2', 'KMD_C2H4O', 'z*_C2H4O', '#raw_pks'])
    
    CF2_series = CF2_series.iloc[:, :-5]
    CF2_series = CF2_series.sort_values(['KMD_CF2_new', 'z*_CF2'])
    CF2_groups_num = CF2_series.groupby(['KMD_CF2_new', 'z*_CF2']).ngroup().max() + 1
    CF2_series = CF2_series.drop(columns=['KMD_CF2_new'])
    # CF2_series.insert(10, '(CF2)n', np.where(CF2_series['DBE'] >= 0, round((CF2_series['C']-1)/2), round((CF2_series['C']-6)/2)))
    
    CF2_series = CF2_series.reset_index(drop=True)
    CF2_series.loc[0, 'stats'] = 'CF2 series:'
    CF2_series.loc[1, 'stats'] = 'Cat A:'
    CF2_series.loc[2, 'stats'] = 'Cat B:'
    CF2_series.loc[3, 'stats'] = 'Cat C:'
    CF2_series.loc[4, 'stats'] = 'Cat D:'
    CF2_series.loc[5, 'stats'] = 'Cat E:'
    
    CF2_series.loc[0, '#_pks'] = int(CF2_series.loc[:, 'exper_mz'].nunique())
    if CF2_series.loc[0, '#_pks'] == 0:
        CF2_series.loc[0, '#_assgns'] = 0
    else:
        CF2_series.loc[0, '#_assgns'] = int(len(CF2_series.index))
    try:
        CF2_series.loc[0, '#_series'] = int(CF2_groups_num)
        CF2_series.loc[1, '#_series'] = int(number_of_A)
        CF2_series.loc[2, '#_series'] = int(number_of_B)
        CF2_series.loc[3, '#_series'] = int(number_of_C)
        CF2_series.loc[4, '#_series'] = int(number_of_D)
        CF2_series.loc[5, '#_series'] = int(number_of_E)
    except ValueError as e:
        if 'cannot convert float NaN to integer' in str(e):
            CF2_series.loc[0, '#_series'] = 0
            
        
    """6. Write the results into an excel file"""
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        CF2_series.to_excel(writer, sheet_name='CF2_series', index=False)
        # CH2_series.to_excel(writer, sheet_name='CH2_series', index=False)
        # C2H4O_series.to_excel(writer, sheet_name='C2H4O_series', index=False)


/var/folders/gg/9t2489gd6kv3l8nkxgtv9rrc0000gn/T/ipykernel_13552/966314758.py:101: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  CF2_series = CF2_series.groupby(['KMD_CF2', 'z*_CF2', 'H', 'Cl', 'Br', 'P', 'S'], group_keys=False).apply(to_category_E)
